# Dashboard Update

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os 
import pandas as pd
from datetime import date
from datetime import datetime
import time
import calendar
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from google.oauth2 import service_account
from selenium.webdriver.support.ui import WebDriverWait

# variables
browser = webdriver.Chrome()
firstLogin = 'insert email1'
firstPassword = 'insert password1'
secondLogin = 'insert email2'
secondPassword = 'insert password2'
todayTimestamp = calendar.timegm(time.gmtime())
linkSubscriptions1 = f'https://app-vlc.hotmart.com/subscriptions?status%5B0%5D=ACTIVE&status%5B1%5D=DELAYED&status%5B2%5D=CANCELLED_BY_ADMIN&status%5B3%5D=CANCELLED_BY_CUSTOMER&status%5B4%5D=CANCELLED_BY_SELLER&endAccessionDate={todayTimestamp}000&accessionDate=1612148400000'
linkSubscriptions2 = 'https://app-vlc.hotmart.com/subscriptions?status%5B0%5D=ACTIVE&status%5B1%5D=DELAYED&status%5B2%5D=CANCELLED_BY_ADMIN&status%5B3%5D=CANCELLED_BY_CUSTOMER&status%5B4%5D=CANCELLED_BY_SELLER&endAccessionDate=1612062000000&accessionDate=1574823600000'
linkSubscriptions3 = f'https://app-vlc.hotmart.com/subscriptions?status%5B0%5D=ACTIVE&status%5B1%5D=DELAYED&status%5B2%5D=CANCELLED_BY_ADMIN&status%5B3%5D=CANCELLED_BY_CUSTOMER&status%5B4%5D=CANCELLED_BY_SELLER&endAccessionDate={todayTimestamp}000&accessionDate=1548986400000'
linkSales1 = f'https://app-vlc.hotmart.com/sales?transactionStatus%5B0%5D=REFUNDED&productId=696286&endDate={todayTimestamp}000&startDate=1574823600000'
linkSales2 = f'https://app-vlc.hotmart.com/sales?transactionStatus%5B0%5D=REFUNDED&endDate={todayTimestamp}000&startDate=1548986400000'

# open browser
browser.get("https://app-vlc.hotmart.com/login/")
browser.maximize_window()
browser.implicitly_wait(10)

# login in new hotmart account
browser.find_element_by_xpath('//*[@class="hot-form-control auth__form-input pl-5 max-height-56"]').send_keys(firstLogin)
browser.find_element_by_xpath('//*[@type="password"]').send_keys(firstPassword)
browser.find_element_by_xpath('//*[@class="hot-btn w-100 p-3 auth-line-height auth-border-radius font-weight-bold hot-btn--success mt-2"]').click()

# central de colaboradores
browser.find_element_by_xpath('//*[@class="hot-btn hot-btn--link hot-dropdown__toggle hot-dropdown__toggle--split align-items-center"]').click()
time.sleep(1)
browser.find_element_by_xpath('//*[@class="hot-dropdown__item d-flex align-items-center js-login-as-attendance"]').click()

# access link1 with filters
browser.get(linkSubscriptions1)

# close modal
browser.find_element_by_xpath('//*[@data-test-id="onboarding-close"]').click()

# download subscriptions csv 1
browser.find_element_by_xpath('//*[@id="sticky-wrapper"]/div/div[2]/div[3]/button[2]').click()

# access link2 with filters
browser.get(linkSubscriptions2)

# download subscriptions csv 2
browser.find_element_by_xpath('//*[@id="sticky-wrapper"]/div/div[2]/div[3]/button[2]').click()

# access sales link
browser.get(linkSales1)

# download refunds csv
browser.find_element_by_xpath('//*[@id="sticky-wrapper"]/div/div[2]/div[4]/button').click()
browser.find_element_by_xpath('//*[@id="sticky-wrapper"]/div/div[2]/div[4]/div/div[1]/button[2]').click()

# logout
browser.find_element_by_xpath('//*[@id="app-header"]/div[2]/a').click()
browser.find_element_by_xpath('//*[@id="account-menu"]/a').click()
browser.find_element_by_xpath('//*[@id="account-menu"]/div/div/a[3]').click()

# login in old hotmart account
browser.find_element_by_xpath('//*[@class="hot-form-control auth__form-input pl-5 max-height-56"]').send_keys(secondLogin)
browser.find_element_by_xpath('//*[@type="password"]').send_keys(secondPassword)
browser.find_element_by_xpath('//*[@class="hot-btn w-100 p-3 auth-line-height auth-border-radius font-weight-bold hot-btn--success mt-2"]').click()

# access subscriptions link 
browser.get(linkSubscriptions3)

# download subscriptions csv
browser.find_element_by_xpath('//*[@id="sticky-wrapper"]/div/div[2]/div[3]/button[2]').click()

# access sales link
browser.get(linkSales2)

# download refunds csv
browser.find_element_by_xpath('//*[@id="sticky-wrapper"]/div/div[2]/div[4]/button').click()
browser.find_element_by_xpath('//*[@id="sticky-wrapper"]/div/div[2]/div[4]/div/div[1]/button[2]').click()

# close browser
browser.close()


# Move files to new folder


In [ ]:
# set variables
today = date.today()
downloadsPath = r"C:\Users\jheff\Downloads"
filesNewPath = r"C:\Users\jheff\Desktop\Hotmart Files" + f"/{today}"
listDownloadFiles = os.listdir(downloadsPath)
subscPath = f"{filesNewPath}\Subscriptions"
salesPath = f"{filesNewPath}\Sales"
listNewPaths = [subscPath, salesPath]

# create new folders or verify if them alrealdy exists
try:
    os.mkdir(filesNewPath)
    messageNewFolders = ("Nova pasta criada.")
except FileExistsError as e:
    messageNewFolders = ("Nova pasta já existe.")

# create new subfolders or verify if them alrealdy exists
try:
    for item in listNewPaths:
        os.mkdir(item)
    messageNewFolders = ("Nova pasta criada.")
except FileExistsError as e:
    messageNewFolders = ("Nova pasta já existe.")

# search correct files in download folder
for file in listDownloadFiles:
    if ".csv" in file:
        if "subscriber" in file:
            # move to new subscription folder
            os.rename(f"{downloadsPath}/{file}", f"{subscPath}/{file}")
        if "sales_history" in file:
            # move to new sales folder
            os.rename(f"{downloadsPath}/{file}", f"{salesPath}/{file}")

# Create Subscription DataFrame appended

In [ ]:
# set variables
dfSubsc = pd.DataFrame()
listSubscFiles = os.listdir(subscPath)

# read csv and append in a single dataframe
for file in listSubscFiles:
    df = pd.read_csv(subscPath + f"/{file}",
                         sep=';',
                         encoding='utf-8',
                         parse_dates=['Adesão', 'Cancelamento'],
                         dayfirst=True)
    dfSubsc = dfSubsc.append(df, ignore_index=True)

dfSubsc = dfSubsc.sort_values(by='Adesão', ignore_index=True)

# Cria Sales DataFrame appended

In [ ]:
# set variables
dfSales = pd.DataFrame()
listSalesFiles = os.listdir(salesPath)

# read csv and append in a single dataframe
for file in listSalesFiles:
    dfSales = pd.read_csv(salesPath + f"/{file}",
                              sep=';',
                              encoding='utf-8',
                              dtype={'Documento':str},
                              parse_dates=['Data de Venda', 'Data de Confirmação'],
                              dayfirst=True)
    dfSales = dfSales.append(dfSales, ignore_index=True)

dfSales = dfSales.sort_values(by='Data de Venda', ignore_index=True)

# Salva dados em arquivo CSV   

In [ ]:
# save subscriptions csv
dfSubsc.to_csv(subscPath + f"/subscriptions.csv")

# save sales csv
dfSales.to_csv(salesPath + f"/sales.csv")
print("CSV's salvos =D")  

# Update Dashboard with g.sheets API

In [ ]:
# set API configuration
keyLoc = r"C:\Users\jheff\Jupyter\keys.json"
serviceAccountFile = keyLoc
scopes = ['https://www.googleapis.com/auth/spreadsheets']
creds = None
creds = service_account.Credentials.from_service_account_file(
        serviceAccountFile, scopes=scopes)

# The ID spreadsheet.
spreadsheetID = 'insert ID'   
service = build('sheets', 'v4', credentials=creds)

# Call the Sheets API
sheet = service.spreadsheets()
resultSubscriptions = sheet.values().get(spreadsheetId=spreadsheetID, range="Base Assinaturas!A1").execute()
resultRefunds = sheet.values().get(spreadsheetId=spreadsheetID, range="Base Reembolsos!A1").execute()

# Adjustments in dataframe data 
df = dfSubsc.astype('str')
df['Valor'] = df['Valor'].replace('9.9', '9.90')
df['Valor'] = df['Valor'].replace('19.9', '19.90')
df['Valor'] = df['Valor'].replace('14.9', '14.90')
df["Cancelamento"] = df["Cancelamento"].replace('NaT', '')

df2 = dfSales.astype('str')
df2['Preço do Produto'] = df2['Preço do Produto'].replace('9.9', '9.90')
df2['Preço do Produto'] = df2['Preço do Produto'].replace('19.9', '19.90')
df2['Preço do Produto'] = df2['Preço do Produto'].replace('14.9', '14.90')

# Update data in sheets
valueRangeBody = {"values":df.to_numpy().tolist(),"majorDimension": "DIMENSION_UNSPECIFIED"}
valueRangeBody2 = {"values":df2.to_numpy().tolist(),"majorDimension": "DIMENSION_UNSPECIFIED"}

requestSubscriptions = sheet.values().update(spreadsheetId=spreadsheetID, 
                                range="Base Assinaturas!D2", 
                                valueInputOption="USER_ENTERED", 
                                body=valueRangeBody)

requestRefunds = sheet.values().update(spreadsheetId=spreadsheetID, 
                                range="Base Reembolsos!D2", 
                                valueInputOption="USER_ENTERED", 
                                body=valueRangeBody2)

response = requestSubscriptions.execute()
response2 = requestRefunds.execute()